In [ ]:
# !pip install gensim polars
# !pip install fugashi[unidic]
# !python -m unidic download

In [ ]:
# !wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# !tar -xzvf ldcc-20140209.tar.gz

In [25]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 演習
## LDAモデルを使って、類似文書の検索
1. livedoor newsコーパスのカテゴリーごとに８：２の割合でデータセットを学習用・テスト用に分割してください
1. トピック数を2~10の中で、最もパープレキシティの低いLDAモデルを作ってください
1. 各カテゴリーごとのテストデータを入力として使って、LDAモデルによって最も類似度の高い文書を学習データから選定してください。
1. 上記で、選定されたデータが入力したデータと同じカテゴリーかどうかを判定してください。同じカテゴリーの場合は成功とします。
1. カテゴリーごとに成功率を計算してください。

## word2vec/k-meansとLDAの比較
1. 上記で学習させたLDAの各トピックの上位10個ずつ単語を抽出してください。
1. これらの単語をword2vecで単語ベクトルに変換してください。
1. これらの単語ベクトル集合をk-meansでクラスタリングしてください。ただし、k-meansのクラスタ数はLDAのトピック数と同じにしてください。
1. 1で抽出したLDAのトピックの単語集合とk-meansのクラスタの単語集合を比較してください。

In [2]:
import os
import polars as pl

# Load the livedoor news corpus
# path_to_corpus = '../../text'  # 事前にDLして解凍が必要（https://www.rondhuit.com/download.html）
path_to_corpus = './text'  # 事前にDLして解凍が必要（https://www.rondhuit.com/download.html）
data = {}
train_cat, train_url, train_date, train_title, train_documents = [], [], [], [], []
test_cat, test_url, test_date, test_title, test_documents = [], [], [], [], []
for category in os.listdir(path_to_corpus):
    if category in ['CHANGES.txt', 'README.txt']:
        continue
    category_path = os.path.join(path_to_corpus, category)
    for i, file in enumerate(os.listdir(category_path)):
        if file in ['LICENSE.txt']:
            continue
        if i < len(os.listdir(category_path))*0.8:
            file_path = os.path.join(category_path, file)
            with open(file_path, 'r') as f:
                train_cat.append(category_path.split("/")[-1])
                f.readline()  # １行目：記事のURL
                f.readline()  # ２行目：記事の日付
                f.readline()  # ３行目：記事のタイトル
                train_documents.append(f.read())  # ４行目以降：記事の本文
        else:
            file_path = os.path.join(category_path, file)
            with open(file_path, 'r') as f:
                test_cat.append(category_path.split("/")[-1])
                f.readline()  # １行目：記事のURL
                f.readline()  # ２行目：記事の日付
                f.readline()  # ３行目：記事のタイトル
                test_documents.append(f.read())  # ４行目以降：記事の本文


df_train = pl.DataFrame({"CATEGORY": train_cat, "DOCUMENT": train_documents})
df_test = pl.DataFrame({"CATEGORY": test_cat, "DOCUMENT": test_documents})
df_train.write_csv("raw_corpus_train.csv")
df_test.write_csv("raw_corpus_test.csv")

In [3]:
df_train.shape[0] / (df_train.shape[0] + df_test.shape[0])

0.8012759603637845

In [58]:
# with open("raw_corpus_train.csv", 'r') as f:
#     header = f.readline()
#     first_line = f.readline()
#     ddf = pl.DataFrame({header.split(',')[0]:first_line.split(',')[0], header.split(',')[1].strip():''.join(first_line.split(',')[1:])} )
#     print(ddf)
#     for line in f.readlines():
#         ddf = pl.concat([ddf, pl.DataFrame({header.split(',')[0]:line.split(',')[0], header.split(',')[1].strip():''.join(line.split(',')[1:])})])

shape: (1, 2)
┌─────────────┬─────────────────────────────────────────────────────────┐
│ CATEGORY    ┆ DOCUMENT                                                │
│ ---         ┆ ---                                                     │
│ str         ┆ str                                                     │
╞═════════════╪═════════════════════════════════════════════════════════╡
│ movie-enter ┆ "　2005年11月から翌2006年7月まで読売新聞にて連載され... │
└─────────────┴─────────────────────────────────────────────────────────┘


In [4]:
import re
import requests
import polars as pl
from fugashi import Tagger
from gensim.corpora import Dictionary, MmCorpus
from gensim import models
from sklearn.model_selection import train_test_split

class LivedoorCorpus():
    def __init__(self, df):
        self.df = df

        # 全角半角文字以外（記号と数字）を正規表現を使って除去
        pattern = r"[^\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\u20000-\u2ffff\sa-zA-Z]"
        self.raw_documents = [re.sub(pattern, "", text) for text in self.df["DOCUMENT"]]
        # Mecabで分かち書きして、単語に分割
        self.raw_documents = [Tagger('-Owakati').parse(text).split() for text in self.raw_documents]
        # ストップワードの除去
        self.raw_documents = self._rm_stopwords()
        # 1文字は除去
        self.raw_documents = [[word for word in text if len(word) > 1]for text in self.raw_documents]

        self.dictionary = Dictionary(self.raw_documents)

        self.bow = [ self.dictionary.doc2bow(text) for text in self.raw_documents]


    def reset_dict_corpus(self):
        self.dictionary = Dictionary(self.raw_documents)
        self.bow = [ self.dictionary.doc2bow(text) for text in self.raw_documents]

    def print_stats(self):
        print(f"文書数: {self.dictionary.num_docs}, " + f"語彙数: {len(self.dictionary)}")

    def dict_top_n(self, top_n: int):
        most_frequent_ids = (v for v in self.dictionary)
        most_frequent_ids = sorted(most_frequent_ids, key=self.dictionary.dfs.get, reverse=True)
        most_frequent_ids = most_frequent_ids[:top_n]
        return [self.dictionary[idx] for idx in most_frequent_ids]
        
    def _rm_stopwords(self):
        # ストップワードの準備
        stopwords_url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
        r = requests.get(stopwords_url)
        tmp = r.text.split('\r\n')
        stopwords = []
        for i in range(len(tmp)):
            if len(tmp[i]) < 1:
                continue
            stopwords.append(tmp[i])

        return [[word for word in text if not word in stopwords]for text in self.raw_documents]

# CSVが読み込めなかったため、dfを与えるようにしています。
train_corpus = LivedoorCorpus(df_train)
test_corpus = LivedoorCorpus(df_test)

In [5]:
train_corpus.reset_dict_corpus()
train_corpus.print_stats()
train_corpus.dictionary.filter_extremes(10,0.5)
train_corpus.print_stats()
train_bow = [ train_corpus.dictionary.doc2bow(text) for text in train_corpus.raw_documents]
# 辞書はtrain_corpusのものを使います。
test_bow = [ train_corpus.dictionary.doc2bow(text) for text in test_corpus.raw_documents]

文書数: 5903, 語彙数: 64506
文書数: 5903, 語彙数: 11877


In [120]:
train_corpus.dictionary.save('livedoor.dict')
MmCorpus.serialize('./train_bow.mm', train_bow)
MmCorpus.serialize('./test_bow.mm', test_bow)

# メモリ不足や遅れている場合に使用
# dictionary = Dictionary.load('livedoor.dict')
# doc_train = MmCorpus('train_bow.mm')
# doc_test = MmCorpus('test_bow.mm')

2023-01-27 03:05:11,416 : INFO : Dictionary lifecycle event {'fname_or_handle': 'livedoor.dict', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-01-27T03:05:11.414109', 'gensim': '4.3.0', 'python': '3.9.6 (default, Sep 26 2022, 11:37:49) \n[Clang 14.0.0 (clang-1400.0.29.202)]', 'platform': 'macOS-12.5.1-arm64-arm-64bit', 'event': 'saving'}
2023-01-27 03:05:11,660 : INFO : saved livedoor.dict
2023-01-27 03:05:11,661 : INFO : storing corpus in Matrix Market format to ./train_bow.mm
2023-01-27 03:05:11,661 : INFO : saving sparse matrix to ./train_bow.mm
2023-01-27 03:05:11,661 : INFO : PROGRESS: saving document #0
2023-01-27 03:05:11,832 : INFO : PROGRESS: saving document #1000
2023-01-27 03:05:11,949 : INFO : PROGRESS: saving document #2000
2023-01-27 03:05:12,063 : INFO : PROGRESS: saving document #3000
2023-01-27 03:05:12,204 : INFO : PROGRESS: saving document #4000
2023-01-27 03:05:12,381 : INFO : PROGRESS: saving document #5000
2023-01-27 03:05:1

In [18]:
topic_range = range(2, 10)

def calc_perplexity(m, c):
    import numpy as np
    return np.exp(-m.log_perplexity(c))

def search_model(corpus_train, corpus_test):
    most = [1.0e6, None]
    print(f"dataset: training/test = {len(corpus_train)}/{len(corpus_test)}")

    for t in topic_range:
        # 辞書はtrain_corpusのものを使います。
        m = models.LdaModel(corpus=corpus_train, id2word=train_corpus.dictionary, num_topics=t, iterations=500, passes=5)
        p1 = calc_perplexity(m, corpus_train)
        p2 = calc_perplexity(m, corpus_test)
        print(f"{t}: perplexity is {p1}/{p2}")
        
        if p2 < most[0]:
            most[0] = p2
            most[1] = m
    
    return most[0], most[1]

perplexity, model_lda = search_model(train_bow, test_bow)
print(f"Best model: topics={model_lda.num_topics}, perplexity={perplexity}")

dataset: training/test = 5903/1464
2: perplexity is 3288.4388309458286/3677.975380846817
3: perplexity is 3187.0854507084205/3692.974381613391
4: perplexity is 2947.683211806905/3566.3653970241276
5: perplexity is 2910.6477622901984/3640.1911254850593
6: perplexity is 2928.6140713124923/3800.1268442208543
7: perplexity is 2758.812666199278/3655.988510858198
8: perplexity is 2823.2095331111045/3822.6017413838867
9: perplexity is 2840.7071136796785/3944.568517727349
Best model: topics=4, perplexity=3566.3653970241276


In [7]:
import numpy as np
from gensim import similarities
index = similarities.MatrixSimilarity(model_lda[train_bow], num_features=len(train_corpus.dictionary))

acc = []
loss = []
for i, doc in enumerate(test_bow):
    similarity_lda = index[doc]
    if df_train[int(np.argmax(similarity_lda))]["CATEGORY"][0] == df_test[i]["CATEGORY"][0]:
        acc.append([df_train[int(np.argmax(similarity_lda))]["DOCUMENT"][0], df_test[i]["DOCUMENT"][0]])
    else:
        loss.append([df_train[int(np.argmax(similarity_lda))]["DOCUMENT"][0], df_test[i]["DOCUMENT"][0]])


In [118]:
len(acc)/(len(acc) + len(loss))

0.09631147540983606

In [27]:
class WVCorpus():
    def __init__(self, corpus):
        self.corpus = corpus
    def __iter__(self):
        return iter(self.corpus)

sentences = WVCorpus(train_corpus.raw_documents)
# instantiating and training the Word2Vec model
model_wv = models.Word2Vec(
    sentences,
    min_count=1,
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42,
)

# getting the training loss value
training_loss = model_wv.get_latest_training_loss()
print(training_loss)

2023-01-27 00:57:12,805 : INFO : collecting all words and their counts
2023-01-27 00:57:12,808 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-01-27 00:57:13,687 : INFO : collected 64506 word types from a corpus of 1657278 raw words and 5903 sentences
2023-01-27 00:57:13,688 : INFO : Creating a fresh vocabulary
2023-01-27 00:57:13,886 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 64506 unique words (100.00% of original 64506, drops 0)', 'datetime': '2023-01-27T00:57:13.886065', 'gensim': '4.3.0', 'python': '3.9.6 (default, Sep 26 2022, 11:37:49) \n[Clang 14.0.0 (clang-1400.0.29.202)]', 'platform': 'macOS-12.5.1-arm64-arm-64bit', 'event': 'prepare_vocab'}
2023-01-27 00:57:13,887 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 1657278 word corpus (100.00% of original 1657278, drops 0)', 'datetime': '2023-01-27T00:57:13.887305', 'gensim': '4.3.0', 'python': '3.9.6 (default, Sep 26 2022, 11:37:49) \n[Clang 1

25655160.0


In [115]:
from sklearn.cluster import KMeans

topic_words = []
for t in range(model_lda.num_topics):
    x = model_lda.show_topic(t, 10)
    topic_words.append([(t, w[0]) for w in x])

topic_df = pl.DataFrame({str(x[0][0]):[word[1] for word in x] for x in topic_words})
topic_words = np.ravel([[word[1] for word in x] for x in topic_words])

word_vectors = [model_wv.wv[x] for x in topic_words]

kmeans_clustering = KMeans(n_clusters = model_lda.num_topics)
idx = kmeans_clustering.fit_predict(word_vectors)

y = []
for t in range(model_lda.num_topics):
    _y = []
    for i, id in enumerate(idx):
        if t == id:
            _y.append((t, topic_words[i]))
    y.append(_y)


cluster_words = {str(w[0][0]):[word[1] for word in w] for w in y}
print(topic_df)
for k,v in cluster_words.items():
    print(k,v)

shape: (10, 4)
┌──────┬──────────────┬──────────┬──────┐
│ 0    ┆ 1            ┆ 2        ┆ 3    │
│ ---  ┆ ---          ┆ ---      ┆ ---  │
│ str  ┆ str          ┆ str      ┆ str  │
╞══════╪══════════════╪══════════╪══════╡
│ 映画 ┆ 日本         ┆ スマート ┆ 女性 │
│ 日本 ┆ 写真         ┆ アプリ   ┆ って │
│ 監督 ┆ キャンペーン ┆ フォン   ┆ たい │
│ 選手 ┆ 応募         ┆ できる   ┆ いい │
│ ...  ┆ ...          ┆ ...      ┆ ...  │
│ 作品 ┆ 発表         ┆ 機能     ┆ だけ │
│ 世界 ┆ より         ┆ AX       ┆ たら │
│ 放送 ┆ 開催         ┆ Android  ┆ たり │
│ 番組 ┆ サイト       ┆ SM       ┆ あり │
└──────┴──────────────┴──────────┴──────┘
0 ['映画', '監督', '公開', '作品', '番組', '写真']
1 ['だっ', '世界', '東京', 'より', 'サイト', 'できる', '女性', 'って', 'たい', 'いい', 'なく', '結婚', 'だけ', 'たら', 'たり', 'あり']
2 ['キャンペーン', '応募', '2012', '発表', '開催', 'スマート', 'アプリ', 'フォン', '対応', '更新', '機能', 'AX', 'Android', 'SM']
3 ['日本', '選手', '放送', '日本']


/Users/hajime/projects/ai_lab/.env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [102]:
topic_df

0,1,2,3
str,str,str,str
"""映画""","""日本""","""スマート""","""女性"""
"""日本""","""写真""","""アプリ""","""って"""
"""監督""","""キャンペーン""","""フォン""","""たい"""
"""選手""","""応募""","""できる""","""いい"""
"""だっ""","""2012""","""対応""","""なく"""
"""公開""","""東京""","""更新""","""結婚"""
"""作品""","""発表""","""機能""","""だけ"""
"""世界""","""より""","""AX""","""たら"""
"""放送""","""開催""","""Android""","""たり"""


In [114]:
for k,v in cluster_words.items():
    print(k,v)

0 ['映画', '監督', '公開', '作品', '番組', '写真']
1 ['日本', '世界', '放送', '日本', 'キャンペーン', '応募', '2012', '東京', '発表', '開催', 'サイト', 'アプリ', '対応', '更新']
2 ['選手', 'だっ', 'より', 'できる', '機能', '女性', 'って', 'たい', 'いい', 'なく', '結婚', 'だけ', 'たら', 'たり', 'あり']
3 ['スマート', 'フォン', 'AX', 'Android', 'SM']
